In [ ]:
## This is the FastAi Baseline for the Flower Recognition Competition on HackerEarth.You can tweak this Notebook or can used the Ideas for your own work.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
print(os.listdir("../input/he_challenge_data/data"))

### Need pretrainedmodels to load the pretrained Cedene models into fastai.
### https://github.com/Cadene/pretrained-models.pytorch

In [ ]:
!pip install pretrainedmodels

In [ ]:
from torchvision.models import *
# import pretrainedmodels

from fastai import *
from fastai.vision import *
from fastai.vision.models import *
from fastai.vision.learner import model_meta
import fastai

from utils import *
import sys
import torch
fastai.__version__

In [ ]:
torch.__version__

In [ ]:
from skimage import io

In [ ]:
io.imshow(io.imread('../input/he_challenge_data/data/train/1100.jpg'))

In [ ]:
lis = os.listdir('../input/he_challenge_data/data/train')

In [ ]:
sub = pd.read_csv('../input/he_challenge_data/data/sample_submission.csv')

In [ ]:
sub.shape

In [ ]:
path = "../input/he_challenge_data/data/train"

In [ ]:
## test filenames to be used to create final submission.
filenames = os.listdir('../input/he_challenge_data/data/test')

In [ ]:
df = pd.read_csv('../input/he_challenge_data/data/train.csv')

In [ ]:
df.head()

In [ ]:
tfms = get_transforms(do_flip=False,flip_vert=False ,max_rotate=10.0, max_zoom=1.22, max_lighting=0.22, max_warp=0.0, p_affine=0.75, p_lighting=0.75)

In [ ]:
data = (ImageList.from_csv(path, csv_name = '../train.csv', suffix='.jpg')
        .split_by_rand_pct(0.1)              
        .label_from_df()            
        .add_test_folder(test_folder = '../test')              
        .transform(tfms, size=299)
        .databunch(bs=32, device= torch.device('cuda:0')).normalize(imagenet_stats))

In [ ]:
## to see the images in train with there labels
data.show_batch(rows=3, figsize=(8,10))

In [ ]:
## print the target classes
print(data.classes)

In [ ]:
#lets create learner. tried with resnet152, densenet201, resnet101
learn1 = cnn_learner(data=data, base_arch=models.resnet152, metrics=[FBeta(beta=1, average='macro'), accuracy],
                    callback_fns=ShowGraph)



In [ ]:
#lets start with steepset slope point. adding wd (weight decay) not to overfit as we are running 15 epochs 
lr = 3e-03
#learn.fit_one_cycle(10, slice(lr))
learn1.fit_one_cycle(15, slice(lr), wd=0.2)

In [ ]:
learn1.freeze_to(-3)

In [ ]:
learn1.fit_one_cycle(3,slice(1e-05,lr/8),wd=0.15)

In [ ]:
learn1.freeze_to(-2)

In [ ]:
learn1.fit_one_cycle(3,slice(1e-06,lr/16),wd=0.1)

In [ ]:
learn1.unfreeze()

In [ ]:
learn1.fit_one_cycle(8,slice(1e-06,lr/16),wd=0.1)

In [ ]:
learn1.export("resnet152.pkl")

In [ ]:
preds1,_ = learn1.TTA(ds_type=DatasetType.Test)

In [ ]:
labelled_preds = []
pred11 = preds1
for pred in pred11:
    labelled_preds.append(int(np.argmax(pred))+1)

submission = pd.DataFrame(
    {'image_id': filenames,
     'category': labelled_preds,
    })
submission['image_id'] = submission['image_id'].apply(lambda x:x.split('.')[0])
submission = submission.sort_values(by = ['image_id'], ascending = [True])
submission.to_csv('submission_resnet152.csv',index=False)

In [ ]:
learn2 = cnn_learner(data=data, base_arch=models.densenet161, metrics=[FBeta(beta=1, average='macro'), accuracy],
                    callback_fns=ShowGraph).mixup()

In [ ]:
#lets start with steepset slope point. adding wd (weight decay) not to overfit as we are running 15 epochs 
lr = 3e-03
#learn.fit_one_cycle(10, slice(lr))
learn2.fit_one_cycle(15, slice(lr), wd=0.2)
learn2.freeze_to(-3)
learn2.fit_one_cycle(3,slice(1e-05,lr/8),wd=0.15)
learn2.freeze_to(-2)
learn2.fit_one_cycle(3,slice(1e-06,lr/16),wd=0.1)
learn2.unfreeze()
learn2.fit_one_cycle(8,slice(1e-06,lr/16),wd=0.1)

In [ ]:
learn2.export("densenet161.pkl")

In [ ]:
preds2,_ = learn2.TTA(ds_type=DatasetType.Test)

In [ ]:
labelled_preds = []
pred12 = preds2
for pred in pred12:
    labelled_preds.append(int(np.argmax(pred))+1)

submission = pd.DataFrame(
    {'image_id': filenames,
     'category': labelled_preds,
    })
submission['image_id'] = submission['image_id'].apply(lambda x:x.split('.')[0])
submission = submission.sort_values(by = ['image_id'], ascending = [True])
submission.to_csv('submission_densenet161.csv',index=False)

In [ ]:
learn3 = cnn_learner(data=data, base_arch=models.densenet201, metrics=[FBeta(beta=1, average='macro'), accuracy],
                    callback_fns=ShowGraph).mixup()

In [ ]:
#lets start with steepset slope point. adding wd (weight decay) not to overfit as we are running 15 epochs 
lr = 3e-03
#learn.fit_one_cycle(10, slice(lr))
learn3.fit_one_cycle(15, slice(lr), wd=0.2)
learn3.freeze_to(-3)
learn3.fit_one_cycle(3,slice(1e-05,lr/8),wd=0.15)
learn3.freeze_to(-2)
learn3.fit_one_cycle(3,slice(1e-06,lr/16),wd=0.1)
learn3.unfreeze()
learn3.fit_one_cycle(8,slice(1e-06,lr/16),wd=0.1)

In [ ]:
learn2.export("densenet201.pkl")

In [ ]:
preds3,_ = learn3.TTA(ds_type=DatasetType.Test)

In [ ]:
labelled_preds = []
pred13 = preds3
for pred in pred13:
    labelled_preds.append(int(np.argmax(pred))+1)

submission = pd.DataFrame(
    {'image_id': filenames,
     'category': labelled_preds,
    })
submission['image_id'] = submission['image_id'].apply(lambda x:x.split('.')[0])
submission = submission.sort_values(by = ['image_id'], ascending = [True])
submission.to_csv('submission_densenet201.csv',index=False)

In [ ]:
## create the submission file 

In [ ]:
labelled_preds = []
pred14 =preds1 + preds2 + preds3
for pred in pred14:
    labelled_preds.append(int(np.argmax(pred))+1)

submission = pd.DataFrame(
    {'image_id': filenames,
     'category': labelled_preds,
    })
submission['image_id'] = submission['image_id'].apply(lambda x:x.split('.')[0])
submission = submission.sort_values(by = ['image_id'], ascending = [True])
submission.to_csv('submission_mix.csv',index=False)

In [ ]:
submission.head()

In [ ]:
## To download the submission file without Commiting the kernel.

In [ ]:
from IPython.display import HTML
import pandas as pd
import numpy as np
import base64

# download it (will only work for files < 2MB or so)
def create_download_link(df, title = "Download CSV file", filename = "submsep_mix.csv"):  
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(submission)

In [ ]:
## This is the Fastai Baseline for the Flower Recognition Competition.If you like the work then plz Upvote the Kernel.
## https://www.hackerearth.com/challenges/competitive/garden-nerd